<a href="https://colab.research.google.com/github/jsKim-prog/AIStudy24/blob/master/GridSearchStudy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 데이터 셋 준비

In [2]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [3]:
# 훈련세트와 테스트세트 분리
# data -> tr_input 80% : ts_input 20%
# tr_input -> sub_input 80% : val_input 20% (검증세트 추가)
from sklearn.model_selection import train_test_split

tr_input, ts_input, tr_target, ts_target = train_test_split(data, target, test_size=0.2)
sub_input, val_input, sub_target, val_target = train_test_split(tr_input, tr_target, test_size=0.2)

print(f"훈련세트/테스트세트 : {tr_input.shape} / {ts_input.shape}")
print(f"훈련세트/검증세트 : {sub_input.shape} / {val_input.shape}")

훈련세트/테스트세트 : (5197, 3) / (1300, 3)
훈련세트/검증세트 : (4157, 3) / (1040, 3)


In [6]:
# 검증 결과
# 훈련 -> 검증세트로 1회 검증 -> 테스트세트로 최종 검증

from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(sub_input, sub_target)
print(f"훈련점수 : {dt.score(sub_input, sub_target):.2%}")  # 훈련점수 : 99.83% -> 과대적합
print(f"검증점수 : {dt.score(val_input, val_target):.2%}")  # 검증점수 : 87.12%


훈련점수 : 99.66%
검증점수 : 86.44%


In [ ]:
print(f"테스트셋점수 : {dt.score(ts_input, ts_target):.2%}")    # 테스트셋점수 : 85.38%

테스트셋점수 : 87.69%


In [ ]:
dt.fit(sub_input, sub_target)
print(f"훈련점수 : {dt.score(sub_input, sub_target):.2%}")  # 훈련점수 : 99.83% -> 변화없음
print(f"검증점수 : {dt.score(val_input, val_target):.2%}")  # 검증점수 : 87.31% -> 조금 상승
print(f"테스트셋점수 : {dt.score(ts_input, ts_target):.2%}")     # 테스트셋점수 : 85.31% -> 조금 떨어짐

훈련점수 : 99.86%
검증점수 : 86.06%
테스트셋점수 : 87.46%


### 교차검증

In [11]:
# tr_input과 ts_input(검증세트를 분리하지 않은 세트)를 사용
from sklearn.model_selection import cross_validate
# 새객체로 새로 훈련
dtc = DecisionTreeClassifier()
dtc.fit(tr_input, tr_target)
# 검증메서드에 삽입
scores = cross_validate(dtc, tr_input, tr_target)
print(scores)
# {'fit_time': array([0.01259494, 0.01222992, 0.01221538, 0.01219225, 0.01187277]),
# 'score_time': array([0.00211406, 0.00214911, 0.00214362, 0.00202274, 0.00206685]),
#'test_score': array([0.86826923, 0.8625    , 0.86236766, 0.83926853, 0.86333013])}

{'fit_time': array([0.01259494, 0.01222992, 0.01221538, 0.01219225, 0.01187277]), 'score_time': array([0.00211406, 0.00214911, 0.00214362, 0.00202274, 0.00206685]), 'test_score': array([0.86826923, 0.8625    , 0.86236766, 0.83926853, 0.86333013])}


In [12]:
# 최종 검증폴드의 점수(test_score 평균)
import numpy as np
print(f"5-폴드 검증결과  : {np.mean(scores['test_score']):.2%}")    # 5-폴드(default) 검증결과  : 85.91%

5-폴드 검증결과  : 85.91%


In [9]:
# cross_validate()는 기본적으로 회귀모델인 KFold 분할기를 사용함
# 분류 모델일 경우 타깃 클래스를 골고루 나누기 위해서 STartfiedKFold를 사용
from sklearn.model_selection import StratifiedKFold # Stratified : 계층화된
scores = cross_validate(dtc, tr_input, tr_target, cv=StratifiedKFold())
print(scores)
print(f"분할기 검증결과  : {np.mean(scores['test_score']):.2%}")
#{'fit_time': array([0.01016831, 0.00873899, 0.00870585, 0.00906849, 0.00926352]),
#'score_time': array([0.00144768, 0.00138354, 0.00143862, 0.00202632, 0.00137281]),
#'test_score': array([0.86634615, 0.85865385, 0.86236766, 0.83830606, 0.86236766])
#분할기 검증결과  : 85.76%

{'fit_time': array([0.01016831, 0.00873899, 0.00870585, 0.00906849, 0.00926352]), 'score_time': array([0.00144768, 0.00138354, 0.00143862, 0.00202632, 0.00137281]), 'test_score': array([0.86634615, 0.85865385, 0.86236766, 0.83830606, 0.86236766])}
분할기 검증결과  : 85.76%


In [10]:
splitter = StratifiedKFold(n_splits=10, shuffle=True)   # 10-폴드 교차검증 수행
scores = cross_validate(dtc, tr_input, tr_target, cv=splitter)
print(scores)
print(f"10-폴드 검증결과  : {np.mean(scores['test_score']):.2%}")
# {'fit_time': array([0.01443315, 0.01351285, 0.01340675, 0.02352142, 0.04588413,0.03307009, 0.02914524, 0.04228973, 0.01724625, 0.01382089]),
#'score_time': array([0.00198936, 0.00187254, 0.00199628, 0.00187755, 0.00197792,0.01891589, 0.009444  , 0.00230503, 0.00196385, 0.00189161]),
#'test_score': array([0.85384615, 0.86730769, 0.85384615, 0.86923077, 0.87692308, 0.875     , 0.85961538, 0.86705202, 0.88439306, 0.86512524])}
#10-폴드 검증결과  : 86.72%

{'fit_time': array([0.01443315, 0.01351285, 0.01340675, 0.02352142, 0.04588413,
       0.03307009, 0.02914524, 0.04228973, 0.01724625, 0.01382089]), 'score_time': array([0.00198936, 0.00187254, 0.00199628, 0.00187755, 0.00197792,
       0.01891589, 0.009444  , 0.00230503, 0.00196385, 0.00189161]), 'test_score': array([0.85384615, 0.86730769, 0.85384615, 0.86923077, 0.87692308,
       0.875     , 0.85961538, 0.86705202, 0.88439306, 0.86512524])}
10-폴드 검증결과  : 86.72%


### 하이퍼파라미터 튜닝
1. 딕셔너리로 매개변수와 탐색할 값을 만듦
2. 결정트리 클래스의 객체를 생성하면서 튜닝 딕셔너리 전달
3. .fit() 하면 전달된 딕셔너리 적용하며 크로스검증 수행

In [13]:
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(3, 20, 1),
          'min_samples_split': range(2, 100, 10)
}

gs = GridSearchCV(DecisionTreeClassifier(), params, n_jobs=-1)
gs.fit(tr_input, tr_target)

print(f"최상의 매개변수 조합 : \n{gs.best_params_}")
print(f"최상의 교차점수 : \n{gs.cv_results_['mean_test_score']}")
# 52.782s
# 최상의 매개변수 조합 :
#{'max_depth': 17, 'min_impurity_decrease': 0.0002, 'min_samples_split': 2}
#최상의 교차점수 :
#[0.83856315 0.83856315 0.83856315 ... 0.85549511 0.85568742 0.85414748]

최상의 매개변수 조합 : 
{'max_depth': 18, 'min_impurity_decrease': 0.00030000000000000003, 'min_samples_split': 2}
최상의 교차점수 : 
[0.84644999 0.84644999 0.84644999 ... 0.86261272 0.86318964 0.86338269]


In [14]:
print(f"최상의 교차점수 : \n{np.max(gs.cv_results_['mean_test_score'])}")

최상의 교차점수 : 
0.8710785148441549


### 랜덤서치
* 매개변수의 값이 아닌 매개변수를 샘플링할 수 있는 확률 분포도 객체 전달
* scipy 사용

In [16]:
from scipy.stats import uniform, randint

rgen = randint(0, 10)   # 0~10 사이에서 샘플링
rgen.rvs(5) # size만큼 샘플링 생성

array([9, 4, 5, 8, 1])

In [17]:
ugen = uniform(0, 1)
ugen.rvs(3)

array([0.73693606, 0.7638266 , 0.45748457])

In [18]:
from sklearn.model_selection import RandomizedSearchCV

params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(10, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25)
}
gs = RandomizedSearchCV(DecisionTreeClassifier(), params, n_iter=100, n_jobs=-1)
# n_iter=100 : 100번 샘플링하여 교차검증 수행
gs.fit(tr_input, tr_target)
print(f"최상의 매개변수 조합 : \n{gs.best_params_}")
print(f"최상의 교차점수 : \n{np.max(gs.cv_results_['mean_test_score'])}")
# 4.853 s -> 빠르다!!

최상의 매개변수 조합 : 
{'max_depth': 15, 'min_impurity_decrease': 0.00033745650738834247, 'min_samples_leaf': 24, 'min_samples_split': 18}
최상의 교차점수 : 
0.8695383875027762


In [19]:
# 테스트 세트 성능 확인
# 최상검증점수는 GridSearchCV(87.10%) 였으나, 수행시간이 너무 길고(53초) RandomizedSearchCV(86.91%, 5초)의 결과와 큰 차이가 없어 RandomizedSearchCV를 최종모델로 결정
dtc=gs.best_estimator_
print(f"테스트세트 점수 : {dtc.score(ts_input, ts_target):.2%}")

테스트세트 점수 : 85.92%


### 결과
* 검증모델 :  RandomizedSearchCV
* best_params
  * 'max_depth': 15,
  * 'min_impurity_decrease': 0.00033745650738834247,
  * 'min_samples_leaf': 24,
  * 'min_samples_split': 18
  * n_iter=100

* 훈련세트 점수 : 86.95%
* 테스트세트 점수 : 85.92%